In [ ]:
!pip install jupyter-dash
!pip install geopandas
!pip install plotly
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install dash==1.19.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python --version

Python 3.7.15


In [ ]:
from jupyter_dash import JupyterDash # Versión de dash para notebooks
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output # Clases Input y Output
import plotly.express as px # Generar gráficas e importar datasets con Plotly

import pandas as pd # Recolección y manipulación de datos
import json

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [ ]:
df=pd.DataFrame({
    "nombre":['art1','art2','art3','art4'],
    "año":[2020,2021,2022,2020],
    "reproducciones": [3,4,8,29],
    "otros":['US','Mx','Es','Us']

})

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/spotify/archive/data_filter.csv')
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65300 entries, 0 to 65299
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   artist   65300 non-null  object 
 1   año      65300 non-null  int64  
 2   streams  65300 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ MB


In [ ]:
df2=pd.read_csv('/content/drive/MyDrive/spotify/archive/data_filter_2.csv')
df2.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
df2['region'].unique()

array(['Global', 'USA', 'MEX', 'BRA', 'GBR', 'DEU', 'JPN', 'FRA', 'ITA',
       'PHL', 'CHL', 'ESP', 'CAN', 'ARG', 'AUT', 'IND', 'SEN', 'TUR',
       'NLD', 'POL', 'DNK', 'NOR', 'COL', 'PER', 'ROU', 'IRL', 'NZL',
       'SGP', 'THA', 'FIN', 'MYS', 'ECU', 'GTM', 'TWN', 'VNM', 'MAR',
       'PRY', 'BEL', 'DOM', 'CRI', 'SWE', 'HKG', 'PRT', 'SLV', 'AUS',
       'CZE', 'HND', 'GRC', 'ISL', 'KOR', 'PAN', 'EGY', 'UKR', 'ZAF',
       'BOL', 'SAU', 'HUN', 'UGA', 'NIC', 'LTU', 'ARE', 'SVK', 'LVA',
       'BGR', 'EST', 'LUX'], dtype=object)

In [ ]:
app=JupyterDash(__name__,external_stylesheets=external_stylesheets)
artistas=df['artist'].unique()


filtered_df2 = df2[df2["año"] == 2017]
filtered_df2 = filtered_df2[filtered_df2['artist']==filtered_df2["artist"][1]]
filtered_df2 = filtered_df2[filtered_df2["region"] != 'Global']

fig2 = px.scatter_geo(filtered_df2, locations="region",size="streams",
                     color="streams",color_continuous_scale=["red", "green", "blue"],
                     projection="natural earth",
                     title='Distribución de reproducciones de {} el año {}'.format('Ed Sheeran' , 2017))


filtered_df3=df2[df2['artist'] == 'Ed Sheeran']
filtered_df3=filtered_df3.groupby(['año'])[['streams']].sum()
filtered_df3.reset_index(inplace=True)
fig3=px.scatter(filtered_df3,x="año",y="streams",color_continuous_scale=["red", "green", "blue"],
                size='streams',
                title='Reproducciones totales de  {} a lo largo de los años'.format('Ed Sheeran'),
                color='streams')

fig3.update_traces(marker=dict(size=25))

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

app.layout=html.Div([
    html.Div([
        html.Div([
            html.H1("Artistas con más reproducciones en Spotify"),
        ], style={'text-align':'center'}),
        
        html.H4("Medina Ortiz Roberto Arturo"),
        html.Div([
        html.A("Linkedin", href='https://www.linkedin.com/in/robert-nidem/', target="_blank"),
        ], style={'background-color':'#A9F344','font-size': '2rem','width':'100px','text-align':'center'}),
    ]),
    html.Div([
        dcc.RadioItems(
            id='scale',
            options=[{'label': i, 'value': i}for i in ['Linear','Log']],
            value='Linear',
            labelStyle={'display': 'inline-block', 'marginTop': '5px'}
        )
    ], style={'background-color':'#9CDE62','display':'inline-block'}),
    html.Div([
      html.Div([
        dcc.Graph(id='graph-rep',
                  hoverData={'points':[{'data':'artista'}]},
                  responsive=True),
        dcc.Slider(
            id='slider-graph',
            min=df['año'].min(),
            max=df['año'].max(),
            value=df['año'].min(),
            marks={str(year): str(year) for year in df['año'].unique()},
            step=None)
      ],style={'width':'100%','display': 'inline-block', 'padding': '0 20'}),
    
      html.Div([
        html.Div(id='display_value', style={'background-color':'#9CDE62'}),
      ],style={'height':'10px'}),
      html.Div([
        html.Div([
          dcc.Graph(id='map-graf',
                    figure=fig2),
        ],style={'background-color':'#9CDE62','display':'inline-block','height':'500px','width':'50%', 'padding': '0 20'}),
        html.Div([
          dcc.Graph(id='stream-graph',
                    figure=fig3),
        ],style={'background-color':'#9CDE62','display':'inline-block','height':'500px','width':'50%'}),
      ]),
    ]),

        
], style={'background-color':'#9CDE62'})

In [ ]:
def create_fig(df, axis_type, title):
  filtered_df = df[df["año"] == 2017].sort_values('streams',ascending=False).head(n=10)
  fig = px.bar(filtered_df, x="streams", y="artist",orientation='h')
  fig.update_xaxes(type='linear' if axis_type == 'Linear' else 'log')
  fig.update_layout(clickmode='event+select')
  fig.update_layout(transition_duration=500)
  return fig


@app.callback(
  Output('graph-rep','figure'),
  [Input('slider-graph','value'),
  Input('scale','value')])
def update_figure(selected_year,xaxis_type):
  filtered_df = df[df["año"] == selected_year].sort_values('streams',ascending=False).head(n=10)

  fig = px.bar(filtered_df, x="streams", y="artist",orientation='h')
  fig.update_layout(autosize=False,
              margin=dict(
                  l=100,
                  r=50,
                  b=50,
                  t=100,
                  pad=4
              ))
  fig.update_xaxes(type='linear' if xaxis_type == 'Linear' else 'log')
  fig.update_yaxes(automargin=True)
  fig.update_layout(title={'text': 'Top 10 artistas más reproducidos en Spotify el año {}'.format(selected_year),
                                            
                          'y':0.9,
                          'x':0.5,
                           'xanchor': 'center',
                           'yanchor': 'top',})
  fig.update_layout(clickmode='event+select')
  fig.update_layout(transition_duration=500)

  return fig

@app.callback(
  Output('display_value','children'),
  Input('graph-rep','clickData'))
def display_map(clickData):
  #return '{}'.format(clickData['points'][0]['y'],)
  #return clickData
  return clickData['points'][0]['y']

@app.callback(
  Output('map-graf','figure'),
  [Input('slider-graph','value'),
  Input('graph-rep','clickData')])
def update_map(selected_year, selected_data):
  filtered_df2 = df2[df2["año"] == selected_year]
  filtered_df2 = filtered_df2[filtered_df2["artist"] == selected_data['points'][0]['y']]
  filtered_df2 = filtered_df2[filtered_df2["region"] != 'Global']

  fig2 = px.scatter_geo(filtered_df2, locations="region",size="streams", 
                     color="streams",color_continuous_scale=["red", "green", "blue"],
                     projection="natural earth",
                     title='Distribución de reproducciones de {} el año {}'.format(selected_data['points'][0]['y'] , selected_year))
    
  #fig2.title('Paises con más reproducciones de {} el año {}'.format(selected_data['points'][0]['y'],selected_year))
  fig2.update_layout(transition_duration=500)
  return fig2

"""@app.callback(
  Output('stream-graph','figure'),
  Input('slider-graph','value'))
def update_stream_map(selected_year):
  filtered_df3=df2[df2['año'] == selected_year]
  filtered_df3=filtered_df3.groupby(['region'])[["streams"]].sum()
  filtered_df3.reset_index(inplace=True)
  filtered_df3=filtered_df3[filtered_df3['region']!='Global']

  fig3 = px.scatter_geo(filtered_df3, locations="region",size="streams",   
                     color="streams",
                     projection="natural earth",
                     title='Reproducciones totales en Spotify el año {}'.format(selected_year))
  fig3.update_layout(transition_duration=500)
  return fig3"""
@app.callback(
    Output('stream-graph','figure'),
    Input('graph-rep','clickData'))
def update_stream_map(data):
  filtered_df3=df2[df2['artist'] == data['points'][0]['y']]
  filtered_df3=filtered_df3.groupby(['año'])[['streams']].sum()
  filtered_df3.reset_index(inplace=True)
  fig3=px.scatter(filtered_df3,x="año",y="streams",color_continuous_scale=["red", "green", "blue"],
                  color='streams',
                  title= 'Reproducciones totales de  {} a lo largo de los años'.format(data['points'][0]['y']))
  
  fig3.update_traces(marker=dict(size=25))
  fig3.update_yaxes(type='log')
  
  fig3.update_layout(transition_duration=500)
  return fig3

if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>